In [67]:
import psycopg2
import config
from configparser import ConfigParser
import pandas as pd

In [68]:
# from configparser import ConfigParser
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:

            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

<h4>Connect DB</h4>

In [69]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        cur.execute('SELECT current_database()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
        # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('LibCS631',)
Database connection closed.


<h4>Creating Tables</h4>

In [50]:

def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
          """
          CREATE TABLE PUBLISHER (
                PUBLISHERID VARCHAR(255) NOT NULL,
                PUBNAME VARCHAR(255),
                ADDRESS VARCHAR(255),
                PRIMARY KEY (PUBLISHERID)
          )
          """,
          """
          CREATE TABLE PERSON (
                PID VARCHAR(75) NOT NULL,
                PNAME VARCHAR(75),
                PRIMARY KEY (PID)
                )
         """
         )
    
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
        print('Query successfully executed')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


if __name__ == '__main__':
    create_tables()

Query successfully executed


In [54]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE DOCUMENT (
            DOCID SERIAL NOT NULL,
            TITLE VARCHAR(255),
            PDATE DATE,
            PUBLISHERID VARCHAR(255),
            PRIMARY KEY (DOCID),
            FOREIGN KEY (PUBLISHERID)
            REFERENCES PUBLISHER (PUBLISHERID)
            ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE BOOK (
                DOCID SERIAL NOT NULL,
                ISBN VARCHAR(255),
                PRIMARY KEY (DOCID),
                FOREIGN KEY (DOCID)
                REFERENCES DOCUMENT (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE JOURNAL_VOLUME (
                DOCID SERIAL NOT NULL,
                VOLUME_NO VARCHAR(75),
                EDITOR VARCHAR(75),
                PRIMARY KEY (DOCID),
                FOREIGN KEY (DOCID)
                REFERENCES DOCUMENT (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (EDITOR)
                REFERENCES PERSON (PID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
        """,
        """
        CREATE TABLE JOURNAL_ISSUE (
                DOCID SERIAL NOT NULL,
                ISSUE_NO VARCHAR(75) NOT NULL,
                SCOPE VARCHAR(75),
                PRIMARY KEY (DOCID, ISSUE_NO),
                FOREIGN KEY (DOCID)
                REFERENCES JOURNAL_VOLUME (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
        """,
        """
        CREATE TABLE PROCEEDINGS (
                DOCID SERIAL NOT NULL,
                CDATE DATE,
                CLOCATION VARCHAR(75),
                CEDITOR VARCHAR(75),
                PRIMARY KEY (DOCID),
                FOREIGN KEY (DOCID)
                REFERENCES DOCUMENT (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
        """,
        """
        CREATE TABLE AUTHORS (
                PID VARCHAR(75) NOT NULL,
                DOCID SERIAL NOT NULL,
                PRIMARY KEY (PID, DOCID),
                FOREIGN KEY (DOCID)
                REFERENCES BOOK (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (PID)
                REFERENCES PERSON (PID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
        """,
        """
        CREATE TABLE GEDITS (
                DOCID SERIAL NOT NULL,
                ISSUE_NO VARCHAR(75) NOT NULL,
                PID VARCHAR(75) NOT NULL,
                PRIMARY KEY (DOCID, ISSUE_NO, PID),
                FOREIGN KEY (PID)
                REFERENCES PERSON (PID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
         """
#               FOREIGN KEY (DOCID)
#               REFERENCES JOURNAL_ISSUE (DOCID)
#               ON UPDATE CASCADE ON DELETE CASCADE, 
#               FOREIGN KEY (ISSUE_NO)
#               REFERENCES JOURNAL_ISSUE (ISSUE_NO)
#               ON UPDATE CASCADE ON DELETE CASCADE,
#         ,        
        """
        CREATE TABLE CHAIRS (
                DOCID SERIAL NOT NULL,
                PID VARCHAR(75) NOT NULL,
                PRIMARY KEY (DOCID, PID),
                FOREIGN KEY (DOCID)
                REFERENCES PROCEEDINGS (DOCID)
                ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (PID)
                REFERENCES PERSON (PID)
                ON UPDATE CASCADE ON DELETE CASCADE
                )
       """
      )
    
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
        print('Query successfully executed')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


if __name__ == '__main__':
    create_tables()

Query successfully executed


<h4>Insert value one row at the time</h4>

In [12]:
# def insert_book():
#     """ insert a new book into the book table """
# #     sql = """INSERT INTO PUBLISHER (PUBLISHERID, PUBNAME, ADDRESS)
# #           VALUES ('980', 'Barnes & Noble', 'Newark'); """
#     sql = """INSERT INTO DOCUMENT (TITLE, PDATE, PUBLISHERID)
#           VALUES ('Early African American Classics (Barnes &amp; Noble Classics Series)', '2/14/1989', '980');"""
#     conn = None
    
#     try:
#         # read database configuration
#         params = config()
#         # connect to the PostgreSQL database
#         conn = psycopg2.connect(**params)
#         # create a new cursor
#         cur = conn.cursor()
#         # execute the INSERT statement
#         cur.execute(sql)
#         # get the generated id back
# #         vendor_id = cur.fetchone()[0]
#         # commit the changes to the database
#         conn.commit()
#         # close communication with the database
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

# insert_book()

<h4>Insert data from csv</h4>

In [55]:

    file = r'publisher.csv'
    sql_insert = """INSERT INTO PUBLISHER (PUBLISHERID, PUBNAME, ADDRESS)
            VALUES(%s, %s, %s)"""

    conn = None
    try:
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()
        with open(file, 'r') as f:
            reader = csv.reader(f)
            next(reader) # This skips the 1st row which is the header.
            for record in reader:
#                 print(record)
                cursor.execute(sql_insert, record)
                conn.commit()
                
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            cursor.close()
            conn.close()
            print("Connection closed.")
        
insert_csv()

Connection closed.


In [56]:
def insert_csv():
    
    file = r'document.csv'
    sql_insert = """INSERT INTO DOCUMENT (TITLE, PDATE, PUBLISHERID)
            VALUES(%s, %s, %s)"""

    conn = None
    try:
        params = config()
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()
        df = pd.read_csv(file, parse_dates=['PDATE'])
            for index, row in df.iterrows():
                cursor.execute(sql_insert, (row['TITLE'], row['PDATE'], row['PUBLISHERID']))
                conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            cursor.close()
            conn.close()
            print("Connection closed.")
            
insert_csv()

Connection closed.
